In [8]:

import pandas as pd
import numpy as np
import json
import geopy
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
import os

In [9]:
with open(r'Resources\data.json') as f:
    locations = json.load(f)

In [11]:
geolocator = Nominatim(user_agent="geo_app")


In [12]:
m = folium.Map(location=[20, 0], zoom_start=2)

In [15]:
with open('Resources\geocoded_data.json', 'r') as f:
    geocoded_data = json.load(f)


In [16]:
gdp_dict = {}
for item in locations:
    country = item['country']
    gdp_dict[country] = {
        'gdp_2018': item.get('gdp_2018', 'N/A'),
        'gdp_2022': item.get('gdp_2022', 'N/A')
    }

merged_data = []
for item in geocoded_data:
    country_name = item['country']
    gdp = gdp_dict.get(country_name, {'gdp_2018': 'N/A', 'gdp_2022': 'N/A'})
    merged_data.append({
        'country': country_name,
        'lat': item['lat'],
        'lon': item['lon'],
        'gdp_2018': gdp['gdp_2018'],
        'gdp_2022': gdp['gdp_2022']
    })
    
# Save the merged data to a new JSON file
with open('merged_data.json', 'w') as f:
    json.dump(merged_data, f, indent=4)


In [18]:
with open('Resources\geocoded_data.json', 'r') as f:
    geocoded_data = json.load(f)
with open('Resources\merged_data.json', 'r') as f:
    merged_data = json.load(f)

In [19]:
gdp_dict = {item['country']: {'gdp_2018': item.get('gdp_2018', 'N/A'), 'gdp_2022': item.get('gdp_2022', 'N/A')} for item in locations}

for item in geocoded_data:
    country = item['country']
    if country in gdp_dict:
        item['gdp_2018'] = gdp_dict[country]['gdp_2018']
        item['gdp_2022'] = gdp_dict[country]['gdp_2022']
    else:
        item['gdp_2018'] = 'N/A'
        item['gdp_2022'] = 'N/A'
with open("geocoded_data_with_gdp.json", 'w') as f:
    json.dump(geocoded_data, f, indent=4)

In [20]:
with open('geocoded_data_with_gdp.json', 'r') as f:
    geocoded_data = json.load(f)

In [21]:
heatmap_layer = folium.FeatureGroup(name='Heatmap')
marker_layer = folium.FeatureGroup(name='Markers')

In [23]:
# Adding markers to the map
for item in geocoded_data:
    if item['lat'] and item['lon']:
        folium.Marker(
            location=[item['lat'], item['lon']],
            popup=(
                f"Country: {item['country']}<br>"
                f"GDP 2018: ${item['gdp_2018']}<br>"
                f"GDP 2022: ${item['gdp_2022']}"
            ),
            icon=folium.Icon(color='blue')
        ).add_to(marker_layer)

# Preparing heatmap data
heatmap_data = []
for item in geocoded_data:
    try:
        lat = float(item['lat'])
        lon = float(item['lon'])
        gdp_2018 = float(item['gdp_2018'])  # Ensure GDP is numeric
        if lat is not None and lon is not None and gdp_2018 is not None:
            heatmap_data.append([lat, lon, gdp_2018])
    except (ValueError, TypeError):
        # Skip invalid data without printing
        pass

# Adding heatmap to the map
if heatmap_data:
    HeatMap(heatmap_data).add_to(heatmap_layer)

# Adding layers and saving the map
heatmap_layer.add_to(m)
marker_layer.add_to(m)
folium.LayerControl().add_to(m)
m.save('gdp.map.html')
